<a href="https://colab.research.google.com/github/404himel/YOLO/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jul  3 15:27:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#importing some libraries for split data
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
#connecting colab to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path_img = "/content/yolo_data/images/train"
train_path_label = "/content/yolo_data/labels/train"
val_path_img = "/content/yolo_data/images/val"
val_path_label = "/content/yolo_data/labels/val"
test_path = "/content/drive/MyDrive/yolov8/Test_images"

# **Train Test Split**

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

def train_test_split(path, train_path_img, train_path_label, val_path_img, val_path_label, test_path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    # Get list of files without extensions
    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')]))

    print(f"--- This folder has a total number of {len(files)} images ---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    # Create required directories
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Copy images and labels to train folder
    for filex in tqdm(files[:train_size]):
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(train_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(train_path_label, filex + '.txt'))

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path) if name.endswith('.jpg')]))
        for filex in tqdm(neg_images):
            shutil.copy2(os.path.join(neg_path, filex + '.jpg'), os.path.join(train_path_img, filex + '.jpg'))

        print(f"------ Total {len(neg_images)} negative images added to the training data -------")
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    # Copy images and labels to validation folder
    for filex in tqdm(files[train_size:]):
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(val_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(val_path_label, filex + '.txt'))

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    # Write train.txt and test.txt files
    with open(os.path.join(test_path, 'train.txt'), 'w') as train_file:
        for filex in files[:train_size]:
            train_file.write(os.path.join(train_path_img, filex + '.jpg') + '\n')

    with open(os.path.join(test_path, 'test.txt'), 'w') as test_file:
        for filex in files[train_size:]:
            test_file.write(os.path.join(val_path_img, filex + '.jpg') + '\n')

    print("------ TASK COMPLETED -------")

# Define paths
train_path_img = "/content/yolo_data/images/train"
train_path_label = "/content/yolo_data/labels/train"
val_path_img = "/content/yolo_data/images/val"
val_path_label = "/content/yolo_data/labels/val"
test_path = "/content/drive/MyDrive/yolov8/Test_images"

# Example usage
train_test_split('/content/drive/MyDrive/yolov8/Data/Football_kaggle', train_path_img, train_path_label, val_path_img, val_path_label, test_path)


------ PROCESS STARTED -------
--- This folder has a total number of 66 images ---


100%|██████████| 53/53 [01:03<00:00,  1.20s/it]


------ Training data created with 80% split 53 images -------


100%|██████████| 13/13 [00:00<00:00, 91.18it/s]


------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.5/78.2 GB disk)


# **CLI**

In [ ]:
#Training
!yolo task = detect mode = train model = yolov8s.pt data = /content/drive/MyDrive/yolov8/data_1.yaml epochs = 10 imgsz = 600 batch=8 project= /content/drive/MyDrive/yolov8/Trainning_result name=football

In [ ]:
#inferencing
!yolo task = detect mode = predict model = /content/drive/MyDrive/yolov8/Trainning_result/football2/weights/best.pt conf=0.55 source = /content/drive/MyDrive/yolov8/Test_images

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/4 /content/drive/MyDrive/yolov8/Test_images/22d_f_135_we.jpg: 352x608 12 players, 94.1ms
image 2/4 /content/drive/MyDrive/yolov8/Test_images/22d_f_180_6ty.jpg: 352x608 15 players, 11.1ms
image 3/4 /content/drive/MyDrive/yolov8/Test_images/we_f_100.jpg: 352x608 7 players, 10.7ms
image 4/4 /content/drive/MyDrive/yolov8/Test_images/we_f_50.jpg: 352x608 12 players, 10.6ms
Speed: 2.9ms preprocess, 31.6ms inference, 151.1ms postprocess per image at shape (1, 3, 352, 608)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/Output

In [ ]:
# inferencing on video
!yolo task = detect mode = predict model = /content/drive/MyDrive/yolov8/Trainning_result/football2/weights/best.pt conf=0.80 source = /content/drive/MyDrive/yolov8/Vid_F

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/2 (frame 1/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 10 players, 103.5ms
video 1/2 (frame 2/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 8 players, 10.8ms
video 1/2 (frame 3/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 7 players, 11.8ms
video 1/2 (frame 4/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 7 players, 10.8ms
video 1/2 (frame 5/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 7 players, 10.8ms
video 1/2 (frame 6/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 8 players, 10.8ms
video 1/2 (frame 7/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 7 players, 12.4ms
video 1/2 (frame 8/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 5 players, 10.7ms
video 1/2 (frame 9/232) /content/drive/MyDrive/yolov8/Vid_F/1.mp4: 352x608 7 players,

In [ ]:
#predict transfer to output
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/Output

In [ ]:
# inferencing on video
!yolo task = detect mode = predict model = /content/drive/MyDrive/yolov8/Trainning_result/football2/weights/best.pt conf=0.80 source = /content/drive/MyDrive/yolov8/Vid_G

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 173.6ms
video 1/1 (frame 2/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 13.5ms
video 1/1 (frame 3/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 17.0ms
video 1/1 (frame 4/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 10.7ms
video 1/1 (frame 5/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 19.9ms
video 1/1 (frame 6/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 20.7ms
video 1/1 (frame 7/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 14.3ms
video 1/1 (frame 8/376) /content/drive/MyDrive/yolov8/Vid_G/1_g.mp4: 608x352 (no detections), 14.7ms
video 1/1 (frame 9/376

In [ ]:
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/yolov8/Output